<a href="https://colab.research.google.com/github/PuttupuR/My-Project/blob/main/Capstone2_Phonepe_Pulse_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install the required libraries
!pip install streamlit 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=b5f2f659a97b127767d0e92b0b6f0ed613daccfe969d02a3809d8378c58885cc
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [ ]:
#Clone the data from github
!git clone https://github.com/PhonePe/pulse.git 

Cloning into 'pulse'...
remote: Enumerating objects: 7975, done.
remote: Counting objects: 100% (7975/7975), done.
remote: Compressing objects: 100% (6637/6637), done.
remote: Total 7975 (delta 2465), reused 6527 (delta 1031), pack-reused 0
Receiving objects: 100% (7975/7975), 2.11 MiB | 4.18 MiB/s, done.
Resolving deltas: 100% (2465/2465), done.


In [ ]:
#It will show the current dirrectory 
!pwd

/content


In [ ]:
#Change the directory to root
%cd

/root


In [ ]:
# Import necessary modules
%%writefile app.py
import sqlite3
import os
import json
import pandas as pd
import streamlit as st
import plotly.express as px
from geopy.geocoders import Nominatim

#Set the Title of the Page
st.title("Phonepe_Pulse_data_visulization")

##aggregated_transaction_data- DataSet1

# Define the database name and the table name
database_name = "Phonepe_Pulse_data.db"

table_name = "Agg_transactions"
table_name1 = "Agg_user"
table_name2 = "top_transaction"
table_name3 = "top_user"
table_name4 = "map_transaction"

# Create a connection to the database
conn = sqlite3.connect(database_name)

# Create the transactions table with the desired schema
create_table_query = """
CREATE TABLE IF NOT EXISTS Agg_transaction (
    state_name TEXT,
    years INTEGER,
    quarters TEXT,
    transaction_type TEXT,
    paymentInstruments_count INTEGER,
    paymentInstruments_amount REAL
);
"""
conn.execute(create_table_query)

# Load the data into a DataFrame
folder_path =  "/content/pulse/data/aggregated/transaction/country/india/state"

# Get a list of state names in the folder path
state_names =  os.listdir(folder_path)

# Define empty lists to hold data from the JSON files
state_name = []
years = []
quarters = []
transaction_type = []
paymentInstruments_count = []
paymentInstruments_amount = []


# Loop over each state and year in the folder path

for state in state_names:
  year_path = f"{folder_path}/{state}"
  year_names =  os.listdir(year_path)
  for year in year_names:
    quarter_path = f"{folder_path}/{state}/{year}"
    quarter_names = os.listdir(quarter_path)
    for quarter in quarter_names:
      data_path = f"{folder_path}/{state}/{year}/{quarter}"
     
      # Load data from JSON file into a DataFrame
      df = pd.read_json(data_path) 

      # Append data to corresponding list
      state_name.append(state)  
      years.append(int(year))
      quarters.append(quarter)
      transaction_type.append(df['data'].values[2][1]['name'])
      paymentInstruments_count.append(int(df['data'].values[2][1]['paymentInstruments'][0]['count']))
      paymentInstruments_amount.append(float(df['data'].values[2][1]['paymentInstruments'][0]['amount']))

# Combine lists into a single DataFrame

df_agg_tran = pd.concat([pd.DataFrame(state_name, columns=['state_name']),
                 pd.DataFrame(years, columns=['years']),
                 pd.DataFrame(quarters, columns=['quarters']),
                 pd.DataFrame(transaction_type, columns=['transaction_type']),
                 pd.DataFrame(paymentInstruments_count, columns=['paymentInstruments_count']),
                 pd.DataFrame(paymentInstruments_amount, columns=['paymentInstruments_amount'])], axis=1)


# Insert the data into the transactions table
df_agg_tran.to_sql(table_name, conn, if_exists='append', index=False)


# Create a connection to the database
conn1 = sqlite3.connect(database_name)

# Create the user table with the desired schema
create_table_query1 = """
CREATE TABLE IF NOT EXISTS Agg_users (
    state_name TEXT,
    years INTEGER,
    quarters TEXT,
    registeredUsers INTEGER,
    appOpens REAL
);
"""
conn1.execute(create_table_query1)

folder_path1 =  "/content/pulse/data/aggregated/user/country/india/state"
state_names1 =  os.listdir(folder_path)

state_name1 = []
years1 = []
quarters1 = []
registeredUsers1 = []
appOpens1 = []

# Loop over each state and year in the folder path

for state in state_names1:
  year_path = f"{folder_path1}/{state}"
  year_names =  os.listdir(year_path)
  for year in year_names:
    quarter_path = f"{folder_path1}/{state}/{year}"
    quarter_names = os.listdir(quarter_path)
    for quarter in quarter_names:
      data_path = f"{folder_path1}/{state}/{year}/{quarter}"

      # Load data from JSON file into a DataFrame
      df = pd.read_json(data_path) 
      
      # Append data to corresponding list
      state_name1.append(state)  
      years1.append(int(year))
      quarters1.append(quarter)
      registeredUsers1.append(df['data'].values[0]['registeredUsers'])
      appOpens1.append(df['data'].values[0]['appOpens'])

# Combine lists into a single DataFrame      
df_agg_user = pd.concat([pd.DataFrame(state_name1, columns=['state_name']),
                 pd.DataFrame(years1, columns=['years']),
                 pd.DataFrame(quarters1, columns=['quarters']),
                 pd.DataFrame(registeredUsers1, columns=['registeredUsers']),
                 pd.DataFrame(appOpens1, columns=['appOpens'])], axis=1)

# Insert the data into the transactions table
df_agg_user.to_sql(table_name1, conn1, if_exists='append', index=False)

# Create a connection to the database
conn2 = sqlite3.connect(database_name)

# Create the user table with the desired schema
create_table_query2 = """
CREATE TABLE IF NOT EXISTS top_transactions (
    state_name TEXT,
    years INTEGER,
    quarters TEXT,
    entity_name TEXT,
    count REAL
    amount REAL
);
"""
conn2.execute(create_table_query2)

folder_path2 =  "/content/pulse/data/top/transaction/country/india/state"
state_names2 =  os.listdir(folder_path)

state_name2 = []
years2 = []
quarters2 = []
entity_name2 = []
count2 = []
amount2 = []

# Loop over each state and year in the folder path

for state in state_names2:
  for year in year_names:
    for quarter in quarter_names:
      data_path = f"/content/pulse/data/top/transaction/country/india/state/{state}/{year}/{quarter}"
      
      # Load data from JSON file into a DataFrame
      df = pd.read_json(data_path) 

      # Append data to corresponding list
      state_name2.append(state)  
      years2.append(year)
      quarters2.append(quarter)
      entity_name2.append(df['data'].values[0][0]['entityName'])
      count2.append(df['data'].values[0][0]['metric']['count'])
      amount2.append(df['data'].values[0][0]['metric']['amount'])

# Combine lists into a single DataFrame      
df_top_tran = pd.concat([pd.DataFrame(state_name2, columns=['state_name']),
                 pd.DataFrame(years2, columns=['years']),
                 pd.DataFrame(quarters2, columns=['quarters']),
                 pd.DataFrame(entity_name2, columns=['entity_name']),
                 pd.DataFrame(count2, columns=['count']),
                 pd.DataFrame(amount2, columns=['amount'])], axis=1)


# Insert the data into the transactions table
df_top_tran.to_sql(table_name2, conn2, if_exists='append', index=False)

# Create a connection to the database
conn3 = sqlite3.connect(database_name)

# Create the user table with the desired schema
create_table_query3 = """
CREATE TABLE IF NOT EXISTS top_users (
    state_name TEXT,
    years INTEGER,
    quarters TEXT,
    district TEXT,
    registeredUsers REAL
    
);
"""
conn3.execute(create_table_query3)

folder_path3 =  "/content/pulse/data/top/user/country/india/state"
state_names3 =  os.listdir(folder_path3)

state_name3 = []
years3 = []
quarters3 = []
district3 = []
registeredUsers3 = []

# Loop over each state and year in the folder path

for state in state_names3:
  for year in year_names:
    for quarter in quarter_names:
      data_path = f"/content/pulse/data/top/user/country/india/state/{state}/{year}/{quarter}"
      
      # Load data from JSON file into a DataFrame
      df = pd.read_json(data_path) 

      # Append data to corresponding list
      state_name3.append(state)  
      years3.append(year)
      quarters3.append(quarter)
      district3.append(df['data'].values[0][0]['name'])
      registeredUsers3.append(df['data'].values[0][0]['registeredUsers'])
      
# Combine lists into a single DataFrame      
df_top_user = pd.concat([pd.DataFrame(state_name3, columns=['state_name']),
                 pd.DataFrame(years3, columns=['years']),
                 pd.DataFrame(quarters3, columns=['quarters']),
                 pd.DataFrame(district3, columns=['district']),
                 pd.DataFrame(registeredUsers3, columns=['registeredUsers'])], axis=1)

# Insert the data into the transactions table
df_top_user.to_sql(table_name3, conn3, if_exists='append', index=False)

# Create a connection to the database
conn4 = sqlite3.connect(database_name)

# Create the user table with the desired schema
create_table_query4 = """
CREATE TABLE IF NOT EXISTS map_transactions (
    state_name TEXT,
    years INTEGER,
    quarters TEXT,
    district TEXT,
    count REAL
    amount REAL
);
"""
conn4.execute(create_table_query4)

folder_path4 =  "/content/pulse/data/map/transaction/hover/country/india/state"
state_names4 =  os.listdir(folder_path4)

state_name4 = []
years4 = []
quarters4 = []
district4 = []
count4 = []
amount4 = []

# Loop over each state and year in the folder path

for state in state_names4:
  for year in year_names:
    for quarter in quarter_names:
      data_path = f"/content/pulse/data/map/transaction/hover/country/india/state/{state}/{year}/{quarter}"
      
      # Load data from JSON file into a DataFrame
      df = pd.read_json(data_path) 

      # Append data to corresponding list
      state_name4.append(state)  
      years4.append(year)
      quarters4.append(quarter)
      district4.append(df['data'].values[0][0]['name'])
      count4.append(df['data'].values[0][0]['metric'][0]['count'])
      amount4.append(df['data'].values[0][0]['metric'][0]['amount'])

# Combine lists into a single DataFrame      
df_map_tran = pd.concat([pd.DataFrame(state_name4, columns=['state_name']),
                 pd.DataFrame(years4, columns=['years']),
                 pd.DataFrame(quarters4, columns=['quarters']),
                 pd.DataFrame(district4, columns=['district']),
                 pd.DataFrame(count4, columns=['count']),
                 pd.DataFrame(amount4, columns=['amount'])], axis=1)

# Insert the data into the transactions table
df_map_tran.to_sql(table_name4, conn4, if_exists='append', index=False)

# Load datasets into dataframes
df1 = df_agg_tran
df1_x_col = "paymentInstruments_count"
df1_y_col = "paymentInstruments_amount"

df2 = df_agg_user
df2_x_col = "quarters"
df2_y_col = "registeredUsers"

df3 = df_top_tran
df3_x_col = "count"
df3_y_col = "amount"

df4 = df_top_user
df4_x_col = "district"
df4_y_col = "registeredUsers"

df5 = df_map_tran
df5_x_col = "district"
df5_y_col = "count"

# Create a Streamlit sidebar to select the dataset
data_selection = st.sidebar.selectbox("Select dataset", ["Dataset 1", "Dataset 2","Dataset 3","Dataset 4","Dataset 5"])

# Filter the selected dataset and column names
if data_selection == "Dataset 1":
    df = df1
    x_col = df1_x_col
    y_col = df1_y_col
elif data_selection == "Dataset 2":
    df = df2
    x_col = df2_x_col
    y_col = df2_y_col
elif data_selection == "Dataset 3":
    df = df3
    x_col = df3_x_col
    y_col = df3_y_col 
elif data_selection == "Dataset 4":
    df = df4
    x_col = df4_x_col
    y_col = df4_y_col         
else:
    df = df5
    x_col = df5_x_col
    y_col = df5_y_col

# Create a Streamlit dropdown menu to select the type of plot or chart to display
chart_type = st.sidebar.selectbox('Please choose the option that corresponds to how you would like the data to be displayed.',
             ['Line chart','Pie Chart','Bar Chart','Scatter Plot','Box Plot','Violin Plot','Histogram'])
 
st.write('You selected:', chart_type)


# Create the chart based on the selected chart type and display it in Streamlit
if chart_type == "Line chart":
    fig = px.line(df, x=x_col, y=y_col,color="state_name")
elif chart_type == "Pie Chart":
    fig = px.pie(df, values=x_col, names=y_col, color="state_name")
elif chart_type == "Bar Chart":
    fig = px.bar(df, x=x_col, y=y_col,color="state_name")
elif chart_type == "Scatter Plot":
    fig = px.scatter(df, x=x_col, y=y_col,color="state_name")  
elif chart_type == "Box Plot":
    fig = px.box(df, x=x_col, y=y_col,color="state_name") 
elif chart_type == "Violin Plot":
    fig = px.violin(df, x=x_col, y=y_col,color="state_name") 
elif chart_type == "Histogram":
    fig = px.histogram(df, x=x_col, y=y_col,color="state_name")                        
st.plotly_chart(fig)

# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc",timeout = 100)

for i in state_name:
  # entering the location name
  getLoc = loc.geocode(i)
  # printing address
  st.write(getLoc.address)
  #printing latitude and longitude
  st.write("Latitude = ", getLoc.latitude, "\n")
  st.write("Longitude = ", getLoc.longitude)
# Close the database connection
conn.close()
conn1.close()    
conn2.close()
conn3.close()
conn4.close()

Overwriting app.py


In [ ]:
# Run Streamlit app in the background using the "run" command and the name of the file
# Use npx localtunnel to create a public URL for the Streamlit app running on port 8501
# This will allow others to access the app from the internet

!streamlit run app.py & npx localtunnel --port 8501 

[############......] - finalize:yargs: sill finalize /root/.npm/_npx/5805/lib/n

npx: installed 22 in 2.73s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.120.99:8501

your url is: https://good-days-stand-34-75-120-99.loca.lt
  Stopping...
^C
